# Indicadores de Segregación Funcional

Estos indicadores permiten identificar usuarios del sistema SAP ERP, que en ambiente de producción (lugar donde se encuentran los datos operativos de la organización), presentan incompatibilidad en sus funciones o asignación de acciones críticas en los permisos asignados para realizar sus labores diarias, por lo cual deben ser monitoreadas semanalmente y reportados trimestralmente en un informe de Auditoría Continua.

Estos indicadores se construyen como variantes que deben ejecutarse en SAP, las cuales comprueban dos tipos de segregación de funciones:

 - Combinaciones de funciones críticas
 - Autorizaciones críticas

El siguiente código es para ejecutar todos los indicadores de segregación funcional **a excepción de los indicadores IS20, IS21 e IS26**, que tienen una lógica distinta.

 - ***Fecha creación: 02.09.2020***
 - ***Fecha modificación_1: 11.09.2020***
 - ***Fecha modificación_2: 30.09.2020***
 - ***Fecha modificación_3: 02.10.2020***
 - ***Fecha modificación_4: 18.12.2020***

## Importar librerías
---

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

### Utilidades

In [2]:
def clean_blankspace(df):
    #eliminar espacios en nombres de columnas
    columns = list(df.columns)
    columns = [w.replace(' ','') for w in columns]
    df.columns = columns
    #Eliminar espacios en cada celda del dataframe
    for columna in list(df.columns):
        if df.dtypes[columna] == np.dtype('O'):
            df[columna] = df[columna].str.replace(' ','')
    return df

def clean_columns(txt):
    df = txt.drop(columns=txt.columns[0:1]).drop(columns=txt.columns[-1]).dropna(how='all').reset_index(drop=True)
    return df

def num_format(df,col):
    df[col] = df[col].str.replace('.','').str.replace(',','.').astype(float)
    if (col=='DMBTR') or (col=='ERFMG') or (col=='VERPR'):
        df[col] = df[col]*100
    return df

def val(row):
    if Fecha > row['Valideza']:
        val = 'CADUCADOS'
    else:
        val = 'ACTIVOS'
    return val

## Cargar tablas
---

### Combinaciones críticas

In [3]:
ZBC_BC_BC_01 = pd.read_csv('ZBC_BC_BC_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_02 = pd.read_csv('ZBC_BC_BC_02.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_03 = pd.read_csv('ZBC_BC_BC_03.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_04 = pd.read_csv('ZBC_BC_BC_04.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_05 = pd.read_csv('ZBC_BC_BC_05.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_06 = pd.read_csv('ZBC_BC_BC_06.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_07 = pd.read_csv('ZBC_BC_BC_07.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_08 = pd.read_csv('ZBC_BC_BC_08.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZBC_BC_BC_09 = pd.read_csv('ZBC_BC_BC_09.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_MA_01 = pd.read_csv('ZFI_EF_MA_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_PA_01 = pd.read_csv('ZFI_EF_PA_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_PA_02 = pd.read_csv('ZFI_EF_PA_02.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_PA_03 = pd.read_csv('ZFI_EF_PA_03.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_PA_04 = pd.read_csv('ZFI_EF_PA_04.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_PA_05 = pd.read_csv('ZFI_EF_PA_05.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_EF_RM_01 = pd.read_csv('ZFI_EF_RM_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZFI_MA_PA_01 = pd.read_csv('ZFI_MA_PA_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZMM_OC_OL_01 = pd.read_csv('ZMM_OC_OL_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZMM_OC_RS_01 = pd.read_csv('ZMM_OC_RS_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
ZMM_SL_OL_01 = pd.read_csv('ZMM_SL_OL_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')

### Autorizaciones críticas

In [4]:
Z_ZF_BC_12 = pd.read_csv('Z_ZF_BC_12.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_BC_13 = pd.read_csv('Z_ZF_BC_13.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_BC_14 = pd.read_csv('Z_ZF_BC_14.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_HR_01 = pd.read_csv('Z_ZF_HR_01.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_HR_02 = pd.read_csv('Z_ZF_HR_02.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_HR_03 = pd.read_csv('Z_ZF_HR_03.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_HR_04 = pd.read_csv('Z_ZF_HR_04.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_HR_05 = pd.read_csv('Z_ZF_HR_05.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_HR_06 = pd.read_csv('Z_ZF_HR_06.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_EF_09 = pd.read_csv('Z_ZF_EF_09.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_EF_10 = pd.read_csv('Z_ZF_EF_10.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1')
Z_ZF_EF_13 = pd.read_csv('Z_ZF_EF_13.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14 = pd.read_csv('Z_ZF_EF_14.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15 = pd.read_csv('Z_ZF_EF_15.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16 = pd.read_csv('Z_ZF_EF_16.txt', sep='|', skiprows=[0,1,2,3,4,5,6,7,8,9], encoding='latin1') # 18.12.2020 nuevas reglas 4Q-2020

### Base con infomación histórica

In [5]:
base_cc = pd.read_excel('Monitoreo IS.xlsx', sheet_name='Combinaciones críticas')
base_ac = pd.read_excel('Monitoreo IS.xlsx', sheet_name='Autorizaciones críticas')
detalle_des = pd.read_excel('Monitoreo IS.xlsx', sheet_name='Detalle Desviaciones')

## Limpieza de tablas
---
####  Combinaciones críticas
##### Eliminar filas y columnas NaN

In [6]:
ZBC_BC_BC_01 = clean_columns(ZBC_BC_BC_01)
ZBC_BC_BC_02 = clean_columns(ZBC_BC_BC_02)
ZBC_BC_BC_03 = clean_columns(ZBC_BC_BC_03)
ZBC_BC_BC_04 = clean_columns(ZBC_BC_BC_04)
ZBC_BC_BC_05 = clean_columns(ZBC_BC_BC_05)
ZBC_BC_BC_06 = clean_columns(ZBC_BC_BC_06)
ZBC_BC_BC_07 = clean_columns(ZBC_BC_BC_07)
ZBC_BC_BC_08 = clean_columns(ZBC_BC_BC_08)
ZBC_BC_BC_09 = clean_columns(ZBC_BC_BC_09)
ZFI_EF_MA_01 = clean_columns(ZFI_EF_MA_01)
ZFI_EF_PA_01 = clean_columns(ZFI_EF_PA_01)
ZFI_EF_PA_02 = clean_columns(ZFI_EF_PA_02)
ZFI_EF_PA_03 = clean_columns(ZFI_EF_PA_03)
ZFI_EF_PA_04 = clean_columns(ZFI_EF_PA_04)
ZFI_EF_PA_05 = clean_columns(ZFI_EF_PA_05)
ZFI_EF_RM_01 = clean_columns(ZFI_EF_RM_01)
ZFI_MA_PA_01 = clean_columns(ZFI_MA_PA_01)
ZMM_OC_OL_01 = clean_columns(ZMM_OC_OL_01)
ZMM_OC_RS_01 = clean_columns(ZMM_OC_RS_01)
ZMM_SL_OL_01 = clean_columns(ZMM_SL_OL_01)

##### Eliminar espacios en nombres de columnas y cada celda

In [7]:
ZBC_BC_BC_01 = clean_blankspace(ZBC_BC_BC_01)
ZBC_BC_BC_02 = clean_blankspace(ZBC_BC_BC_02)
ZBC_BC_BC_03 = clean_blankspace(ZBC_BC_BC_03)
ZBC_BC_BC_04 = clean_blankspace(ZBC_BC_BC_04)
ZBC_BC_BC_05 = clean_blankspace(ZBC_BC_BC_05)
ZBC_BC_BC_06 = clean_blankspace(ZBC_BC_BC_06)
ZBC_BC_BC_07 = clean_blankspace(ZBC_BC_BC_07)
ZBC_BC_BC_08 = clean_blankspace(ZBC_BC_BC_08)
ZBC_BC_BC_09 = clean_blankspace(ZBC_BC_BC_09)
ZFI_EF_MA_01 = clean_blankspace(ZFI_EF_MA_01)
ZFI_EF_PA_01 = clean_blankspace(ZFI_EF_PA_01)
ZFI_EF_PA_02 = clean_blankspace(ZFI_EF_PA_02)
ZFI_EF_PA_03 = clean_blankspace(ZFI_EF_PA_03)
ZFI_EF_PA_04 = clean_blankspace(ZFI_EF_PA_04)
ZFI_EF_PA_05 = clean_blankspace(ZFI_EF_PA_05)
ZFI_EF_RM_01 = clean_blankspace(ZFI_EF_RM_01)
ZFI_MA_PA_01 = clean_blankspace(ZFI_MA_PA_01)
ZMM_OC_OL_01 = clean_blankspace(ZMM_OC_OL_01)
ZMM_OC_RS_01 = clean_blankspace(ZMM_OC_RS_01)
ZMM_SL_OL_01 = clean_blankspace(ZMM_SL_OL_01)

***Reemplazar valores de Valideza del año 9999***

In [8]:
hoy = datetime.now().strftime('%d.%m.%Y')

In [9]:
ZBC_BC_BC_01['Valideza'] = ZBC_BC_BC_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_02['Valideza'] = ZBC_BC_BC_02.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_03['Valideza'] = ZBC_BC_BC_03.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_04['Valideza'] = ZBC_BC_BC_04.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_05['Valideza'] = ZBC_BC_BC_05.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_06['Valideza'] = ZBC_BC_BC_06.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_07['Valideza'] = ZBC_BC_BC_07.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_08['Valideza'] = ZBC_BC_BC_08.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZBC_BC_BC_09['Valideza'] = ZBC_BC_BC_09.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_MA_01['Valideza'] = ZFI_EF_MA_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_PA_01['Valideza'] = ZFI_EF_PA_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_PA_02['Valideza'] = ZFI_EF_PA_02.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_PA_03['Valideza'] = ZFI_EF_PA_03.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_PA_04['Valideza'] = ZFI_EF_PA_04.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_PA_05['Valideza'] = ZFI_EF_PA_05.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_EF_RM_01['Valideza'] = ZFI_EF_RM_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZFI_MA_PA_01['Valideza'] = ZFI_MA_PA_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZMM_OC_OL_01['Valideza'] = ZMM_OC_OL_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZMM_OC_RS_01['Valideza'] = ZMM_OC_RS_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
ZMM_SL_OL_01['Valideza'] = ZMM_SL_OL_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)

***Transformar los valores de Valideza de str en float***

In [10]:
ZBC_BC_BC_01 = num_format(ZBC_BC_BC_01,'Valideza')
ZBC_BC_BC_02 = num_format(ZBC_BC_BC_02,'Valideza')
ZBC_BC_BC_03 = num_format(ZBC_BC_BC_03,'Valideza')
ZBC_BC_BC_04 = num_format(ZBC_BC_BC_04,'Valideza')
ZBC_BC_BC_05 = num_format(ZBC_BC_BC_05,'Valideza')
ZBC_BC_BC_06 = num_format(ZBC_BC_BC_06,'Valideza')
ZBC_BC_BC_07 = num_format(ZBC_BC_BC_07,'Valideza')
ZBC_BC_BC_08 = num_format(ZBC_BC_BC_08,'Valideza')
ZBC_BC_BC_09 = num_format(ZBC_BC_BC_09,'Valideza')
ZFI_EF_MA_01 = num_format(ZFI_EF_MA_01,'Valideza')
ZFI_EF_PA_01 = num_format(ZFI_EF_PA_01,'Valideza')
ZFI_EF_PA_02 = num_format(ZFI_EF_PA_02,'Valideza')
ZFI_EF_PA_03 = num_format(ZFI_EF_PA_03,'Valideza')
ZFI_EF_PA_04 = num_format(ZFI_EF_PA_04,'Valideza')
ZFI_EF_PA_05 = num_format(ZFI_EF_PA_05,'Valideza')
ZFI_EF_RM_01 = num_format(ZFI_EF_RM_01,'Valideza')
ZFI_MA_PA_01 = num_format(ZFI_MA_PA_01,'Valideza')
ZMM_OC_OL_01 = num_format(ZMM_OC_OL_01,'Valideza')
ZMM_OC_RS_01 = num_format(ZMM_OC_RS_01,'Valideza')
ZMM_SL_OL_01 = num_format(ZMM_SL_OL_01,'Valideza')

***Transformar a formato fecha la columna Valideza***

In [11]:
ZBC_BC_BC_01['Valideza'] = pd.to_datetime(ZBC_BC_BC_01['Valideza'], format='%d%m%Y')
ZBC_BC_BC_02['Valideza'] = pd.to_datetime(ZBC_BC_BC_02['Valideza'], format='%d%m%Y')
ZBC_BC_BC_03['Valideza'] = pd.to_datetime(ZBC_BC_BC_03['Valideza'], format='%d%m%Y')
ZBC_BC_BC_04['Valideza'] = pd.to_datetime(ZBC_BC_BC_04['Valideza'], format='%d%m%Y')
ZBC_BC_BC_05['Valideza'] = pd.to_datetime(ZBC_BC_BC_05['Valideza'], format='%d%m%Y')
ZBC_BC_BC_06['Valideza'] = pd.to_datetime(ZBC_BC_BC_06['Valideza'], format='%d%m%Y')
ZBC_BC_BC_07['Valideza'] = pd.to_datetime(ZBC_BC_BC_07['Valideza'], format='%d%m%Y')
ZBC_BC_BC_08['Valideza'] = pd.to_datetime(ZBC_BC_BC_08['Valideza'], format='%d%m%Y')
ZBC_BC_BC_09['Valideza'] = pd.to_datetime(ZBC_BC_BC_09['Valideza'], format='%d%m%Y')
ZFI_EF_MA_01['Valideza'] = pd.to_datetime(ZFI_EF_MA_01['Valideza'], format='%d%m%Y')
ZFI_EF_PA_01['Valideza'] = pd.to_datetime(ZFI_EF_PA_01['Valideza'], format='%d%m%Y')
ZFI_EF_PA_02['Valideza'] = pd.to_datetime(ZFI_EF_PA_02['Valideza'], format='%d%m%Y')
ZFI_EF_PA_03['Valideza'] = pd.to_datetime(ZFI_EF_PA_03['Valideza'], format='%d%m%Y')
ZFI_EF_PA_04['Valideza'] = pd.to_datetime(ZFI_EF_PA_04['Valideza'], format='%d%m%Y')
ZFI_EF_PA_05['Valideza'] = pd.to_datetime(ZFI_EF_PA_05['Valideza'], format='%d%m%Y')
ZFI_EF_RM_01['Valideza'] = pd.to_datetime(ZFI_EF_RM_01['Valideza'], format='%d%m%Y')
ZFI_MA_PA_01['Valideza'] = pd.to_datetime(ZFI_MA_PA_01['Valideza'], format='%d%m%Y')
ZMM_OC_OL_01['Valideza'] = pd.to_datetime(ZMM_OC_OL_01['Valideza'], format='%d%m%Y')
ZMM_OC_RS_01['Valideza'] = pd.to_datetime(ZMM_OC_RS_01['Valideza'], format='%d%m%Y')
ZMM_SL_OL_01['Valideza'] = pd.to_datetime(ZMM_SL_OL_01['Valideza'], format='%d%m%Y')

***Crear columna para verificar grupo***

In [12]:
Fecha = date.today()

In [13]:
ZBC_BC_BC_01['Estado_calculado'] = ZBC_BC_BC_01.apply(val, axis=1)
ZBC_BC_BC_02['Estado_calculado'] = ZBC_BC_BC_02.apply(val, axis=1)
ZBC_BC_BC_03['Estado_calculado'] = ZBC_BC_BC_03.apply(val, axis=1)
ZBC_BC_BC_04['Estado_calculado'] = ZBC_BC_BC_04.apply(val, axis=1)
ZBC_BC_BC_05['Estado_calculado'] = ZBC_BC_BC_05.apply(val, axis=1)
ZBC_BC_BC_06['Estado_calculado'] = ZBC_BC_BC_06.apply(val, axis=1)
ZBC_BC_BC_07['Estado_calculado'] = ZBC_BC_BC_07.apply(val, axis=1)
ZBC_BC_BC_08['Estado_calculado'] = ZBC_BC_BC_08.apply(val, axis=1)
ZBC_BC_BC_09['Estado_calculado'] = ZBC_BC_BC_09.apply(val, axis=1)
ZFI_EF_MA_01['Estado_calculado'] = ZFI_EF_MA_01.apply(val, axis=1)
ZFI_EF_PA_01['Estado_calculado'] = ZFI_EF_PA_01.apply(val, axis=1)
ZFI_EF_PA_02['Estado_calculado'] = ZFI_EF_PA_02.apply(val, axis=1)
ZFI_EF_PA_03['Estado_calculado'] = ZFI_EF_PA_03.apply(val, axis=1)
ZFI_EF_PA_04['Estado_calculado'] = ZFI_EF_PA_04.apply(val, axis=1)
ZFI_EF_PA_05['Estado_calculado'] = ZFI_EF_PA_05.apply(val, axis=1)
ZFI_EF_RM_01['Estado_calculado'] = ZFI_EF_RM_01.apply(val, axis=1)
ZFI_MA_PA_01['Estado_calculado'] = ZFI_MA_PA_01.apply(val, axis=1)
ZMM_OC_OL_01['Estado_calculado'] = ZMM_OC_OL_01.apply(val, axis=1)
ZMM_OC_RS_01['Estado_calculado'] = ZMM_OC_RS_01.apply(val, axis=1)
ZMM_SL_OL_01['Estado_calculado'] = ZMM_SL_OL_01.apply(val, axis=1)

##### Eliminar columnas que no son de interés y valores duplicados

In [14]:
ZBC_BC_BC_01 = ZBC_BC_BC_01.drop(['IDdeCC','TextoCC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_02 = ZBC_BC_BC_02.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_03 = ZBC_BC_BC_03.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_04 = ZBC_BC_BC_04.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_05 = ZBC_BC_BC_05.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_06 = ZBC_BC_BC_06.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_07 = ZBC_BC_BC_07.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_08 = ZBC_BC_BC_08.drop(['IDdeCC','Textodelacombinacióncrítica(CC)','Valideza','Grupo','Válidode','Nºliquid.'],axis=1).drop_duplicates()
ZBC_BC_BC_09 = ZBC_BC_BC_09.drop(['IDdeCC','Textodelacombinacióncrítica(CC)','Valideza','Grupo','Válidode','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_MA_01 = ZFI_EF_MA_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_PA_01 = ZFI_EF_PA_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_PA_02 = ZFI_EF_PA_02.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_PA_03 = ZFI_EF_PA_03.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_PA_04 = ZFI_EF_PA_04.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_PA_05 = ZFI_EF_PA_05.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_EF_RM_01 = ZFI_EF_RM_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZFI_MA_PA_01 = ZFI_MA_PA_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZMM_OC_OL_01 = ZMM_OC_OL_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZMM_OC_RS_01 = ZMM_OC_RS_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
ZMM_SL_OL_01 = ZMM_SL_OL_01.drop(['IDdeCC','Textocombin.crítica','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()

#### Autoizaciones críticas
##### Eliminar filas y columnas NaN

In [15]:
Z_ZF_BC_12 = clean_columns(Z_ZF_BC_12)
Z_ZF_BC_13 = clean_columns(Z_ZF_BC_13)
Z_ZF_BC_14 = clean_columns(Z_ZF_BC_14)
Z_ZF_HR_01 = clean_columns(Z_ZF_HR_01)
Z_ZF_HR_02 = clean_columns(Z_ZF_HR_02)
Z_ZF_HR_03 = clean_columns(Z_ZF_HR_03)
Z_ZF_HR_04 = clean_columns(Z_ZF_HR_04)
Z_ZF_HR_05 = clean_columns(Z_ZF_HR_05)
Z_ZF_HR_06 = clean_columns(Z_ZF_HR_06)
Z_ZF_EF_09 = clean_columns(Z_ZF_EF_09)
Z_ZF_EF_10 = clean_columns(Z_ZF_EF_10)
Z_ZF_EF_13 = clean_columns(Z_ZF_EF_13) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14 = clean_columns(Z_ZF_EF_14) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15 = clean_columns(Z_ZF_EF_15) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16 = clean_columns(Z_ZF_EF_16) # 18.12.2020 nuevas reglas 4Q-2020

##### Eliminar espacios en nombres de columnas y cada celda

In [16]:
Z_ZF_BC_12 = clean_blankspace(Z_ZF_BC_12)
Z_ZF_BC_13 = clean_blankspace(Z_ZF_BC_13)
Z_ZF_BC_14 = clean_blankspace(Z_ZF_BC_14)
Z_ZF_HR_01 = clean_blankspace(Z_ZF_HR_01)
Z_ZF_HR_02 = clean_blankspace(Z_ZF_HR_02)
Z_ZF_HR_03 = clean_blankspace(Z_ZF_HR_03)
Z_ZF_HR_04 = clean_blankspace(Z_ZF_HR_04)
Z_ZF_HR_05 = clean_blankspace(Z_ZF_HR_05)
Z_ZF_HR_06 = clean_blankspace(Z_ZF_HR_06)
Z_ZF_EF_09 = clean_blankspace(Z_ZF_EF_09)
Z_ZF_EF_10 = clean_blankspace(Z_ZF_EF_10)
Z_ZF_EF_13 = clean_blankspace(Z_ZF_EF_13) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14 = clean_blankspace(Z_ZF_EF_14) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15 = clean_blankspace(Z_ZF_EF_15) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16 = clean_blankspace(Z_ZF_EF_16) # 18.12.2020 nuevas reglas 4Q-2020

***Reemplazar valores de Valideza del año 9999***

In [17]:
Z_ZF_BC_12['Valideza'] = Z_ZF_BC_12.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_BC_13['Valideza'] = Z_ZF_BC_13.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_BC_14['Valideza'] = Z_ZF_BC_14.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_HR_01['Valideza'] = Z_ZF_HR_01.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_HR_02['Valideza'] = Z_ZF_HR_02.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_HR_03['Valideza'] = Z_ZF_HR_03.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_HR_04['Valideza'] = Z_ZF_HR_04.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_HR_05['Valideza'] = Z_ZF_HR_05.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_HR_06['Valideza'] = Z_ZF_HR_06.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_EF_09['Valideza'] = Z_ZF_EF_09.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_EF_10['Valideza'] = Z_ZF_EF_10.Valideza.str.replace(r'(^.*9999.*$)', hoy)
Z_ZF_EF_13['Valideza'] = Z_ZF_EF_13.Valideza.str.replace(r'(^.*9999.*$)', hoy) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14['Valideza'] = Z_ZF_EF_14.Valideza.str.replace(r'(^.*9999.*$)', hoy) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15['Valideza'] = Z_ZF_EF_15.Valideza.str.replace(r'(^.*9999.*$)', hoy) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16['Valideza'] = Z_ZF_EF_16.Valideza.str.replace(r'(^.*9999.*$)', hoy) # 18.12.2020 nuevas reglas 4Q-2020

***Transformar los valores de Valideza de str en float***

In [18]:
Z_ZF_BC_12 = num_format(Z_ZF_BC_12,'Valideza')
Z_ZF_BC_13 = num_format(Z_ZF_BC_13,'Valideza')
Z_ZF_BC_14 = num_format(Z_ZF_BC_14,'Valideza')
Z_ZF_HR_01 = num_format(Z_ZF_HR_01,'Valideza')
Z_ZF_HR_02 = num_format(Z_ZF_HR_02,'Valideza')
Z_ZF_HR_03 = num_format(Z_ZF_HR_03,'Valideza')
Z_ZF_HR_04 = num_format(Z_ZF_HR_04,'Valideza')
Z_ZF_HR_05 = num_format(Z_ZF_HR_05,'Valideza')
Z_ZF_HR_06 = num_format(Z_ZF_HR_06,'Valideza')
Z_ZF_EF_09 = num_format(Z_ZF_EF_09,'Valideza')
Z_ZF_EF_10 = num_format(Z_ZF_EF_10,'Valideza')
Z_ZF_EF_13 = num_format(Z_ZF_EF_13,'Valideza') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14 = num_format(Z_ZF_EF_14,'Valideza') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15 = num_format(Z_ZF_EF_15,'Valideza') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16 = num_format(Z_ZF_EF_16,'Valideza') # 18.12.2020 nuevas reglas 4Q-2020

***Transformar a formato fecha la columna Valideza***

In [19]:
Z_ZF_BC_12['Valideza'] = pd.to_datetime(Z_ZF_BC_12['Valideza'], format='%d%m%Y')
Z_ZF_BC_13['Valideza'] = pd.to_datetime(Z_ZF_BC_13['Valideza'], format='%d%m%Y')
Z_ZF_BC_14['Valideza'] = pd.to_datetime(Z_ZF_BC_14['Valideza'], format='%d%m%Y')
Z_ZF_HR_01['Valideza'] = pd.to_datetime(Z_ZF_HR_01['Valideza'], format='%d%m%Y')
Z_ZF_HR_02['Valideza'] = pd.to_datetime(Z_ZF_HR_02['Valideza'], format='%d%m%Y')
Z_ZF_HR_03['Valideza'] = pd.to_datetime(Z_ZF_HR_03['Valideza'], format='%d%m%Y')
Z_ZF_HR_04['Valideza'] = pd.to_datetime(Z_ZF_HR_04['Valideza'], format='%d%m%Y')
Z_ZF_HR_05['Valideza'] = pd.to_datetime(Z_ZF_HR_05['Valideza'], format='%d%m%Y')
Z_ZF_HR_06['Valideza'] = pd.to_datetime(Z_ZF_HR_06['Valideza'], format='%d%m%Y')
Z_ZF_EF_09['Valideza'] = pd.to_datetime(Z_ZF_EF_09['Valideza'], format='%d%m%Y')
Z_ZF_EF_10['Valideza'] = pd.to_datetime(Z_ZF_EF_10['Valideza'], format='%d%m%Y')
Z_ZF_EF_13['Valideza'] = pd.to_datetime(Z_ZF_EF_13['Valideza'], format='%d%m%Y') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14['Valideza'] = pd.to_datetime(Z_ZF_EF_14['Valideza'], format='%d%m%Y') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15['Valideza'] = pd.to_datetime(Z_ZF_EF_15['Valideza'], format='%d%m%Y') # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16['Valideza'] = pd.to_datetime(Z_ZF_EF_16['Valideza'], format='%d%m%Y') # 18.12.2020 nuevas reglas 4Q-2020

***Crear columna para verificar grupo***

In [20]:
Z_ZF_BC_12['Estado_calculado'] = Z_ZF_BC_12.apply(val, axis=1)
Z_ZF_BC_13['Estado_calculado'] = Z_ZF_BC_13.apply(val, axis=1)
Z_ZF_BC_14['Estado_calculado'] = Z_ZF_BC_14.apply(val, axis=1)
Z_ZF_HR_01['Estado_calculado'] = Z_ZF_HR_01.apply(val, axis=1)
Z_ZF_HR_02['Estado_calculado'] = Z_ZF_HR_02.apply(val, axis=1)
Z_ZF_HR_03['Estado_calculado'] = Z_ZF_HR_03.apply(val, axis=1)
Z_ZF_HR_04['Estado_calculado'] = Z_ZF_HR_04.apply(val, axis=1)
Z_ZF_HR_05['Estado_calculado'] = Z_ZF_HR_05.apply(val, axis=1)
Z_ZF_HR_06['Estado_calculado'] = Z_ZF_HR_06.apply(val, axis=1)
Z_ZF_EF_09['Estado_calculado'] = Z_ZF_EF_09.apply(val, axis=1)
Z_ZF_EF_10['Estado_calculado'] = Z_ZF_EF_10.apply(val, axis=1)
Z_ZF_EF_13['Estado_calculado'] = Z_ZF_EF_13.apply(val, axis=1) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_14['Estado_calculado'] = Z_ZF_EF_14.apply(val, axis=1) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_15['Estado_calculado'] = Z_ZF_EF_15.apply(val, axis=1) # 18.12.2020 nuevas reglas 4Q-2020
Z_ZF_EF_16['Estado_calculado'] = Z_ZF_EF_16.apply(val, axis=1) # 18.12.2020 nuevas reglas 4Q-2020

##### Eliminar columnas que no son de interés y valores duplicados

In [21]:
Z_ZF_BC_12 = Z_ZF_BC_12.drop(['IDdeAC','TextoAC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_BC_13 = Z_ZF_BC_13.drop(['IDdeAC','TextoAC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_BC_14 = Z_ZF_BC_14.drop(['IDdeAC','TextoAC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_HR_01 = Z_ZF_HR_01.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_HR_02 = Z_ZF_HR_02.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_HR_03 = Z_ZF_HR_03.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_HR_04 = Z_ZF_HR_04.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_HR_05 = Z_ZF_HR_05.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_HR_06 = Z_ZF_HR_06.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_EF_09 = Z_ZF_EF_09.drop(['IDdeAC','TextoAC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_EF_10 = Z_ZF_EF_10.drop(['IDdeAC','TextoAC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_EF_13 = Z_ZF_EF_13.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_EF_14 = Z_ZF_EF_14.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_EF_15 = Z_ZF_EF_15.drop(['IDdeAC','Textoautoriz.crít.','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()
Z_ZF_EF_16 = Z_ZF_EF_16.drop(['IDdeAC','TextoAC','Válidode','Valideza','Grupo','Nºliquid.'],axis=1).drop_duplicates()

## Ejecución de indicadores
---

In [22]:
Fecha = date.today().strftime("%d-%m-%y")

### Combinaciones críticas

**ZBC_BC_BC_01**

In [23]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_01 = ZBC_BC_BC_01.loc[(ZBC_BC_BC_01['Tipo']!='SServ.') & (ZBC_BC_BC_01['Estado_calculado']!='CADUCADOS')]

In [24]:
#Cuento los usuarios que quedan
cuenta1 = len(ZBC_BC_BC_01.index)

In [25]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta1]
                                
ZBC_BC_BC_01_r = pd.DataFrame({'Variante':'ZBC_BC_BC_01','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_01_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_01,06-10-20,0


**ZBC_BC_BC_02**

In [26]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_02 = ZBC_BC_BC_02.loc[(ZBC_BC_BC_02['Tipo']!='SServ.') & (ZBC_BC_BC_02['Estado_calculado']!='CADUCADOS')]

In [27]:
#Cuento los usuarios que quedan
cuenta2 = len(ZBC_BC_BC_02.index)

In [28]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta2]
                                
ZBC_BC_BC_02_r = pd.DataFrame({'Variante':'ZBC_BC_BC_02','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_02_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_02,06-10-20,0


**ZBC_BC_BC_03**

In [29]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_03 = ZBC_BC_BC_03.loc[(ZBC_BC_BC_03['Tipo']!='SServ.') & (ZBC_BC_BC_03['Estado_calculado']!='CADUCADOS')]

In [30]:
#Cuento los usuarios que quedan
cuenta3 = len(ZBC_BC_BC_03.index)

In [31]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta3]
                                
ZBC_BC_BC_03_r = pd.DataFrame({'Variante':'ZBC_BC_BC_03','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_03_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_03,06-10-20,0


**ZBC_BC_BC_04**

In [32]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_04 = ZBC_BC_BC_04.loc[(ZBC_BC_BC_04['Tipo']!='SServ.') & (ZBC_BC_BC_04['Estado_calculado']!='CADUCADOS')]

In [33]:
#Cuento los usuarios que quedan
cuenta4 = len(ZBC_BC_BC_04.index)

In [34]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta4]
                                
ZBC_BC_BC_04_r = pd.DataFrame({'Variante':'ZBC_BC_BC_04','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_04_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_04,06-10-20,0


**ZBC_BC_BC_05**

In [35]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_05 = ZBC_BC_BC_05.loc[(ZBC_BC_BC_05['Tipo']!='SServ.') & (ZBC_BC_BC_05['Estado_calculado']!='CADUCADOS')]

In [36]:
#Cuento los usuarios que quedan
cuenta5 = len(ZBC_BC_BC_05.index)

In [37]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta5]
                                
ZBC_BC_BC_05_r = pd.DataFrame({'Variante':'ZBC_BC_BC_05','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_05_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_05,06-10-20,0


**ZBC_BC_BC_06**

In [38]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_06 = ZBC_BC_BC_06.loc[(ZBC_BC_BC_06['Tipo']!='SServ.') & (ZBC_BC_BC_06['Estado_calculado']!='CADUCADOS')]

In [39]:
#Cuento los usuarios que quedan
cuenta6 = len(ZBC_BC_BC_06.index)

In [40]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta6]
                                
ZBC_BC_BC_06_r = pd.DataFrame({'Variante':'ZBC_BC_BC_06','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_06_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_06,06-10-20,0


**ZBC_BC_BC_07**

In [41]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_07 = ZBC_BC_BC_07.loc[(ZBC_BC_BC_07['Tipo']!='SServ.') & (ZBC_BC_BC_07['Estado_calculado']!='CADUCADOS')]

In [42]:
#Cuento los usuarios que quedan
cuenta7 = len(ZBC_BC_BC_07.index)

In [43]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta7]
                                
ZBC_BC_BC_07_r = pd.DataFrame({'Variante':'ZBC_BC_BC_07','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_07_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_07,06-10-20,0


**ZBC_BC_BC_08**

In [44]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_08 = ZBC_BC_BC_08.loc[(ZBC_BC_BC_08['Tipo']!='SServ.') & (ZBC_BC_BC_08['Estado_calculado']!='CADUCADOS')]

In [45]:
#Cuento los usuarios que quedan
cuenta8 = len(ZBC_BC_BC_08.index)

In [46]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta8]
                                
ZBC_BC_BC_08_r = pd.DataFrame({'Variante':'ZBC_BC_BC_08','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_08_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_08,06-10-20,0


**ZBC_BC_BC_09**

In [47]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZBC_BC_BC_09 = ZBC_BC_BC_09.loc[(ZBC_BC_BC_09['Tipo']!='SServ.') & (ZBC_BC_BC_09['Estado_calculado']!='CADUCADOS')]

In [48]:
#Cuento los usuarios que quedan
cuenta9 = len(ZBC_BC_BC_09.index)

In [49]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta9]
                                
ZBC_BC_BC_09_r = pd.DataFrame({'Variante':'ZBC_BC_BC_09','Fecha': Fecha,'Desviaciones': cuenta})
ZBC_BC_BC_09_r                                         

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_09,06-10-20,0


**ZFI_EF_MA_01**

In [50]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_MA_01 = ZFI_EF_MA_01.loc[(ZFI_EF_MA_01['Tipo']!='SServ.') & (ZFI_EF_MA_01['Estado_calculado']!='CADUCADOS')]

In [51]:
#Cuento los usuarios que quedan
cuenta10 = len(ZFI_EF_MA_01.index)

In [52]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta10]
                                
ZFI_EF_MA_01_r = pd.DataFrame({'Variante':'ZFI_EF_MA_01','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_MA_01_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_MA_01,06-10-20,0


**ZFI_EF_PA_01**

In [53]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_PA_01 = ZFI_EF_PA_01.loc[(ZFI_EF_PA_01['Tipo']!='SServ.') & (ZFI_EF_PA_01['Estado_calculado']!='CADUCADOS')]

In [54]:
#Cuento los usuarios que quedan
cuenta11 = len(ZFI_EF_PA_01.index)

In [55]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta11]
                                
ZFI_EF_PA_01_r = pd.DataFrame({'Variante':'ZFI_EF_PA_01','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_PA_01_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_PA_01,06-10-20,1


**ZFI_EF_PA_02**

In [56]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_PA_02 = ZFI_EF_PA_02.loc[(ZFI_EF_PA_02['Tipo']!='SServ.') & (ZFI_EF_PA_02['Estado_calculado']!='CADUCADOS')]

In [57]:
#Cuento los usuarios que quedan
cuenta12 = len(ZFI_EF_PA_02.index)

In [58]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta12]
                                
ZFI_EF_PA_02_r = pd.DataFrame({'Variante':'ZFI_EF_PA_02','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_PA_02_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_PA_02,06-10-20,2


**ZFI_EF_PA_03**

In [59]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_PA_03 = ZFI_EF_PA_03.loc[(ZFI_EF_PA_03['Tipo']!='SServ.') & (ZFI_EF_PA_03['Estado_calculado']!='CADUCADOS')]

In [60]:
#Cuento los usuarios que quedan
cuenta13 = len(ZFI_EF_PA_03.index)

In [61]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta13]
                                
ZFI_EF_PA_03_r = pd.DataFrame({'Variante':'ZFI_EF_PA_03','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_PA_03_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_PA_03,06-10-20,2


**ZFI_EF_PA_04**

In [62]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_PA_04 = ZFI_EF_PA_04.loc[(ZFI_EF_PA_04['Tipo']!='SServ.') & (ZFI_EF_PA_04['Estado_calculado']!='CADUCADOS')]

In [63]:
#Cuento los usuarios que quedan
cuenta14 = len(ZFI_EF_PA_04.index)

In [64]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta14]
                                
ZFI_EF_PA_04_r = pd.DataFrame({'Variante':'ZFI_EF_PA_04','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_PA_04_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_PA_04,06-10-20,0


**ZFI_EF_PA_05**

In [65]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_PA_05 = ZFI_EF_PA_05.loc[(ZFI_EF_PA_05['Tipo']!='SServ.') & (ZFI_EF_PA_05['Estado_calculado']!='CADUCADOS')]

In [66]:
#Cuento los usuarios que quedan
cuenta15 = len(ZFI_EF_PA_05.index)

In [67]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta15]
                                
ZFI_EF_PA_05_r = pd.DataFrame({'Variante':'ZFI_EF_PA_05','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_PA_05_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_PA_05,06-10-20,27


**ZFI_EF_RM_01**

In [68]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_EF_RM_01 = ZFI_EF_RM_01.loc[(ZFI_EF_RM_01['Tipo']!='SServ.') & (ZFI_EF_RM_01['Estado_calculado']!='CADUCADOS')]

In [69]:
#Cuento los usuarios que quedan
cuenta16 = len(ZFI_EF_RM_01.index)

In [70]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta16]
                                
ZFI_EF_RM_01_r = pd.DataFrame({'Variante':'ZFI_EF_RM_01','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_EF_RM_01_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_EF_RM_01,06-10-20,8


**ZFI_MA_PA_01**

In [71]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZFI_MA_PA_01 = ZFI_MA_PA_01.loc[(ZFI_MA_PA_01['Tipo']!='SServ.') & (ZFI_MA_PA_01['Estado_calculado']!='CADUCADOS')]

In [72]:
#Cuento los usuarios que quedan
cuenta17 = len(ZFI_MA_PA_01.index)

In [73]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta17]
                                
ZFI_MA_PA_01_r = pd.DataFrame({'Variante':'ZFI_MA_PA_01','Fecha': Fecha,'Desviaciones': cuenta})
ZFI_MA_PA_01_r                                         

,Variante,Fecha,Desviaciones
0,ZFI_MA_PA_01,06-10-20,0


**ZMM_OC_OL_01**

In [74]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZMM_OC_OL_01 = ZMM_OC_OL_01.loc[(ZMM_OC_OL_01['Tipo']!='SServ.') & (ZMM_OC_OL_01['Estado_calculado']!='CADUCADOS')]

In [75]:
#Cuento los usuarios que quedan
cuenta18 = len(ZMM_OC_OL_01.index)

In [76]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta18]
                                
ZMM_OC_OL_01_r = pd.DataFrame({'Variante':'ZMM_OC_OL_01','Fecha': Fecha,'Desviaciones': cuenta})
ZMM_OC_OL_01_r                                         

,Variante,Fecha,Desviaciones
0,ZMM_OC_OL_01,06-10-20,6


**ZMM_OC_RS_01**

In [77]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZMM_OC_RS_01 = ZMM_OC_RS_01.loc[(ZMM_OC_RS_01['Tipo']!='SServ.') & (ZMM_OC_RS_01['Estado_calculado']!='CADUCADOS')]

In [78]:
#Cuento los usuarios que quedan
cuenta19 = len(ZMM_OC_RS_01.index)

In [79]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta19]
                                
ZMM_OC_RS_01_r = pd.DataFrame({'Variante':'ZMM_OC_RS_01','Fecha': Fecha,'Desviaciones': cuenta})
ZMM_OC_RS_01_r                                         

,Variante,Fecha,Desviaciones
0,ZMM_OC_RS_01,06-10-20,2


**ZMM_SL_OL_01**

In [80]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
ZMM_SL_OL_01 = ZMM_SL_OL_01.loc[(ZMM_SL_OL_01['Tipo']!='SServ.') & (ZMM_SL_OL_01['Estado_calculado']!='CADUCADOS')]

In [81]:
#Cuento los usuarios que quedan
cuenta20 = len(ZMM_SL_OL_01.index)

In [82]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta20]
                                
ZMM_SL_OL_01_r = pd.DataFrame({'Variante':'ZMM_SL_OL_01','Fecha': Fecha,'Desviaciones': cuenta})
ZMM_SL_OL_01_r                                         

,Variante,Fecha,Desviaciones
0,ZMM_SL_OL_01,06-10-20,26


**Unir todos los resultados de combinaciones críticas**
***

In [83]:
comb_criticas = pd.concat([ZBC_BC_BC_01_r,ZBC_BC_BC_02_r,ZBC_BC_BC_03_r,ZBC_BC_BC_04_r,ZBC_BC_BC_05_r,ZBC_BC_BC_06_r,
                           ZBC_BC_BC_07_r,ZBC_BC_BC_08_r,ZBC_BC_BC_09_r,ZFI_EF_MA_01_r,ZFI_EF_PA_01_r,ZFI_EF_PA_02_r,
                           ZFI_EF_PA_03_r,ZFI_EF_PA_04_r,ZFI_EF_PA_05_r,ZFI_EF_RM_01_r,ZFI_MA_PA_01_r,ZMM_OC_OL_01_r,
                           ZMM_OC_RS_01_r,ZMM_SL_OL_01_r],axis=0).reset_index(drop=True)
comb_criticas

,Variante,Fecha,Desviaciones
0,ZBC_BC_BC_01,06-10-20,0
1,ZBC_BC_BC_02,06-10-20,0
2,ZBC_BC_BC_03,06-10-20,0
3,ZBC_BC_BC_04,06-10-20,0
4,ZBC_BC_BC_05,06-10-20,0
5,ZBC_BC_BC_06,06-10-20,0
6,ZBC_BC_BC_07,06-10-20,0
7,ZBC_BC_BC_08,06-10-20,0
8,ZBC_BC_BC_09,06-10-20,0
9,ZFI_EF_MA_01,06-10-20,0


### Autorizaciones críticas

**Z_ZF_BC_12**

In [84]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_BC_12 = Z_ZF_BC_12.loc[(Z_ZF_BC_12['Tipo']!='SServ.') & (Z_ZF_BC_12['Estado_calculado']!='CADUCADOS')]

In [85]:
#Cuento los usuarios que quedan
cuenta21 = len(Z_ZF_BC_12.index)

In [86]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta21]
                                
Z_ZF_BC_12_r = pd.DataFrame({'Variante':'Z_ZF_BC_12','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_BC_12_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_BC_12,06-10-20,0


**Z_ZF_BC_13**

In [87]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_BC_13 = Z_ZF_BC_13.loc[(Z_ZF_BC_13['Tipo']!='SServ.') & (Z_ZF_BC_13['Estado_calculado']!='CADUCADOS')]

In [88]:
#Cuento los usuarios que quedan
cuenta22 = len(Z_ZF_BC_13.index)

In [89]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta22]
                                
Z_ZF_BC_13_r = pd.DataFrame({'Variante':'Z_ZF_BC_13','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_BC_13_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_BC_13,06-10-20,1


**Z_ZF_BC_14**

In [90]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_BC_14 = Z_ZF_BC_14.loc[(Z_ZF_BC_14['Tipo']!='SServ.') & (Z_ZF_BC_14['Estado_calculado']!='CADUCADOS')]

In [91]:
#Cuento los usuarios que quedan
cuenta23 = len(Z_ZF_BC_14.index)

In [92]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta23]
                                
Z_ZF_BC_14_r = pd.DataFrame({'Variante':'Z_ZF_BC_14','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_BC_14_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_BC_14,06-10-20,0


**Z_ZF_HR_01**

In [93]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_HR_01 = Z_ZF_HR_01.loc[(Z_ZF_HR_01['Tipo']!='SServ.') & (Z_ZF_HR_01['Estado_calculado']!='CADUCADOS')]

In [94]:
#Cuento los usuarios que quedan
cuenta24 = len(Z_ZF_HR_01.index)

In [95]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta24]
                                
Z_ZF_HR_01_r = pd.DataFrame({'Variante':'Z_ZF_HR_01','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_HR_01_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_HR_01,06-10-20,47


**Z_ZF_HR_02**

In [96]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_HR_02 = Z_ZF_HR_02.loc[(Z_ZF_HR_02['Tipo']!='SServ.') & (Z_ZF_HR_02['Estado_calculado']!='CADUCADOS')]

In [97]:
#Cuento los usuarios que quedan
cuenta25 = len(Z_ZF_HR_02.index)

In [98]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta25]
                                
Z_ZF_HR_02_r = pd.DataFrame({'Variante':'Z_ZF_HR_02','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_HR_02_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_HR_02,06-10-20,14


**Z_ZF_HR_03**

In [99]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_HR_03 = Z_ZF_HR_03.loc[(Z_ZF_HR_03['Tipo']!='SServ.') & (Z_ZF_HR_03['Estado_calculado']!='CADUCADOS')]

In [100]:
#Cuento los usuarios que quedan
cuenta26 = len(Z_ZF_HR_03.index)

In [101]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta26]
                                
Z_ZF_HR_03_r = pd.DataFrame({'Variante':'Z_ZF_HR_03','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_HR_03_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_HR_03,06-10-20,10


**Z_ZF_HR_04**

In [102]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_HR_04 = Z_ZF_HR_04.loc[(Z_ZF_HR_04['Tipo']!='SServ.') & (Z_ZF_HR_04['Estado_calculado']!='CADUCADOS')]

In [103]:
#Cuento los usuarios que quedan
cuenta27 = len(Z_ZF_HR_04.index)

In [104]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta27]
                                
Z_ZF_HR_04_r = pd.DataFrame({'Variante':'Z_ZF_HR_04','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_HR_04_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_HR_04,06-10-20,13


**Z_ZF_HR_05**

In [105]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_HR_05 = Z_ZF_HR_05.loc[(Z_ZF_HR_05['Tipo']!='SServ.') & (Z_ZF_HR_05['Estado_calculado']!='CADUCADOS')]

In [106]:
#Cuento los usuarios que quedan
cuenta28 = len(Z_ZF_HR_05.index)

In [107]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta28]
                                
Z_ZF_HR_05_r = pd.DataFrame({'Variante':'Z_ZF_HR_05','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_HR_05_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_HR_05,06-10-20,10


**Z_ZF_HR_06**

In [108]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_HR_06 = Z_ZF_HR_06.loc[(Z_ZF_HR_06['Tipo']!='SServ.') & (Z_ZF_HR_06['Estado_calculado']!='CADUCADOS')]

In [109]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_HR_06.index)

In [110]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_HR_06_r = pd.DataFrame({'Variante':'Z_ZF_HR_06','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_HR_06_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_HR_06,06-10-20,13


**Z_ZF_EF_09**

In [111]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_EF_09 = Z_ZF_EF_09.loc[(Z_ZF_EF_09['Tipo']!='SServ.') & (Z_ZF_EF_09['Estado_calculado']!='CADUCADOS')]

In [112]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_EF_09.index)

In [113]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_EF_09_r = pd.DataFrame({'Variante':'Z_ZF_EF_09','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_EF_09_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_EF_09,06-10-20,31


**Z_ZF_EF_10**

In [114]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_EF_10 = Z_ZF_EF_10.loc[(Z_ZF_EF_10['Tipo']!='SServ.') & (Z_ZF_EF_10['Estado_calculado']!='CADUCADOS')]

In [115]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_EF_10.index)

In [116]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_EF_10_r = pd.DataFrame({'Variante':'Z_ZF_EF_10','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_EF_10_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_EF_10,06-10-20,400


**Z_ZF_EF_13**

In [114]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_EF_13 = Z_ZF_EF_13.loc[(Z_ZF_EF_13['Tipo']!='SServ.') & (Z_ZF_EF_13['Estado_calculado']!='CADUCADOS')]

In [115]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_EF_13.index)

In [116]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_EF_13_r = pd.DataFrame({'Variante':'Z_ZF_EF_13','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_EF_13_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_EF_10,06-10-20,400


**Z_ZF_EF_14**

In [114]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_EF_14 = Z_ZF_EF_14.loc[(Z_ZF_EF_14['Tipo']!='SServ.') & (Z_ZF_EF_14['Estado_calculado']!='CADUCADOS')]

In [115]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_EF_14.index)

In [116]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_EF_14_r = pd.DataFrame({'Variante':'Z_ZF_EF_14','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_EF_14_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_EF_10,06-10-20,400


**Z_ZF_EF_15**

In [114]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_EF_15 = Z_ZF_EF_15.loc[(Z_ZF_EF_15['Tipo']!='SServ.') & (Z_ZF_EF_15['Estado_calculado']!='CADUCADOS')]

In [115]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_EF_15.index)

In [116]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_EF_15_r = pd.DataFrame({'Variante':'Z_ZF_EF_15','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_EF_15_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_EF_10,06-10-20,400


**Z_ZF_EF_16**

In [114]:
#Filtro para dejar los que en Tipo son distintos a "SServ." y en Grupo son distintos a "Caducados"
Z_ZF_EF_16 = Z_ZF_EF_16.loc[(Z_ZF_EF_16['Tipo']!='SServ.') & (Z_ZF_EF_16['Estado_calculado']!='CADUCADOS')]

In [115]:
#Cuento los usuarios que quedan
cuenta29 = len(Z_ZF_EF_16.index)

In [116]:
#Creo nuevo DF con el nombre de la variante y cantidad de usuarios
cuenta = [cuenta29]
                                
Z_ZF_EF_16_r = pd.DataFrame({'Variante':'Z_ZF_EF_16','Fecha': Fecha,'Desviaciones': cuenta})
Z_ZF_EF_16_r                                         

,Variante,Fecha,Desviaciones
0,Z_ZF_EF_10,06-10-20,400


**Unir todos los resultados de autorizaciones críticas**
***

In [117]:
autoriz_criticas = pd.concat([Z_ZF_BC_12_r,Z_ZF_BC_13_r,Z_ZF_BC_14_r,Z_ZF_HR_01_r,Z_ZF_HR_02_r,Z_ZF_HR_03_r,Z_ZF_HR_04_r,
                             Z_ZF_HR_05_r,Z_ZF_HR_06_r,Z_ZF_EF_09_r,Z_ZF_EF_10_r,Z_ZF_EF_13_r,Z_ZF_EF_14_r,Z_ZF_EF_15_r,Z_ZF_EF_16_r],axis=0).reset_index(drop=True)
autoriz_criticas

,Variante,Fecha,Desviaciones
0,Z_ZF_BC_12,06-10-20,0
1,Z_ZF_BC_13,06-10-20,1
2,Z_ZF_BC_14,06-10-20,0
3,Z_ZF_HR_01,06-10-20,47
4,Z_ZF_HR_02,06-10-20,14
5,Z_ZF_HR_03,06-10-20,10
6,Z_ZF_HR_04,06-10-20,13
7,Z_ZF_HR_05,06-10-20,10
8,Z_ZF_HR_06,06-10-20,13
9,Z_ZF_EF_09,06-10-20,31


### Agregar las cantidades de combinaciones y autorizaciones críticas por variante a la base histórica

In [118]:
#Limpiamos las tablas bases para eliminar priemra columna
base_cc = base_cc.drop(['Unnamed: 0'],axis=1)
base_ac = base_ac.drop(['Unnamed: 0'],axis=1)
detalle_des = detalle_des.drop(['Unnamed: 0'],axis=1) 

In [119]:
#Creamos DataFrame con las columnas de datos de los indicadores
base_cc_info = base_cc[['Indicador','Variante','Texto']].drop_duplicates()
base_ac_info = base_ac[['Indicador','Variante','Texto']].drop_duplicates()

In [120]:
#Agregamos las combinaciones críticas a su base histórica
resultados_cc = pd.merge(base_cc_info, comb_criticas, on='Variante', how='inner')
base_cc = pd.concat([base_cc, resultados_cc],axis=0).reset_index(drop=True)

base_cc.tail()

,Indicador,Variante,Texto,Fecha,Desviaciones
35,IS06,ZFI_EF_RM_01,RECEPCIONAR MATERIALES Y FACTURAR PAGO (MIGO -...,06-10-20,8
36,IS03,ZFI_MA_PA_01,ACTUALIZAR DATOS BANCO PROVEEDOR E INICIAR PAG...,06-10-20,0
37,IS01,ZMM_OC_OL_01,GESTIONAR COMPRAS Y LIBERARLAS (ME21N-ME29N),06-10-20,6
38,IS05,ZMM_OC_RS_01,GESTIONAR COMPRAS ZCDP Y RECEPCIONAR MATERIAL ...,06-10-20,2
39,IS10,ZMM_SL_OL_01,LIBERAR SOL. DE PEDIDO Y ORDENES DE COMPRA (ME...,06-10-20,26


In [121]:
#Agregamos las autorizaciones críticas a su base histórica
resultados_ac = pd.merge(base_ac_info, autoriz_criticas, on='Variante', how='inner')
base_ac = pd.concat([base_ac, resultados_ac],axis=0).reset_index(drop=True)

base_ac.tail()

,Indicador,Variante,Texto,Fecha,Desviaciones
17,IS30,Z_ZF_HR_04,Critica: Actualizar datos maestros personal (P...,06-10-20,13
18,IS31,Z_ZF_HR_05,Critica: Entrada rápida maestros personal (PA7...,06-10-20,10
19,IS32,Z_ZF_HR_06,Critica: Entrada rápida maestros personal (PA7...,06-10-20,13
20,IS33,Z_ZF_EF_09,Crítica: Anular Compensación (FBRA),06-10-20,31
21,IS34,Z_ZF_EF_10,Crítica: Anular Documento (FB08),06-10-20,400


### Agregamos detalle de las desviaciones encontradas

In [122]:
#Agregamos columnas con nombre del indicador, variante y fecha a cada dataframe con el detalle de las desviaciones

#Combinaciones críticas
ZBC_BC_BC_01[['Indicador','Variante','Fecha']] = 'IS11','ZBC_BC_BC_01',Fecha
ZBC_BC_BC_02[['Indicador','Variante','Fecha']] = 'IS12','ZBC_BC_BC_02',Fecha
ZBC_BC_BC_03[['Indicador','Variante','Fecha']] = 'IS13','ZBC_BC_BC_03',Fecha
ZBC_BC_BC_04[['Indicador','Variante','Fecha']] = 'IS14','ZBC_BC_BC_04',Fecha
ZBC_BC_BC_05[['Indicador','Variante','Fecha']] = 'IS15','ZBC_BC_BC_05',Fecha
ZBC_BC_BC_06[['Indicador','Variante','Fecha']] = 'IS16','ZBC_BC_BC_06',Fecha
ZBC_BC_BC_07[['Indicador','Variante','Fecha']] = 'IS17','ZBC_BC_BC_07',Fecha
ZBC_BC_BC_08[['Indicador','Variante','Fecha']] = 'IS18','ZBC_BC_BC_08',Fecha
ZBC_BC_BC_09[['Indicador','Variante','Fecha']] = 'IS19','ZBC_BC_BC_09',Fecha
ZFI_EF_MA_01[['Indicador','Variante','Fecha']] = 'IS02','ZFI_EF_MA_01',Fecha
ZFI_EF_PA_01[['Indicador','Variante','Fecha']] = 'IS04','ZFI_EF_PA_01',Fecha
ZFI_EF_PA_02[['Indicador','Variante','Fecha']] = 'IS07','ZFI_EF_PA_02',Fecha
ZFI_EF_PA_03[['Indicador','Variante','Fecha']] = 'IS08','ZFI_EF_PA_03',Fecha
ZFI_EF_PA_04[['Indicador','Variante','Fecha']] = 'IS09','ZFI_EF_PA_04',Fecha
ZFI_EF_PA_05[['Indicador','Variante','Fecha']] = 'IS22','ZFI_EF_PA_05',Fecha
ZFI_EF_RM_01[['Indicador','Variante','Fecha']] = 'IS06','ZFI_EF_RM_01',Fecha
ZFI_MA_PA_01[['Indicador','Variante','Fecha']] = 'IS03','ZFI_MA_PA_01',Fecha
ZMM_OC_OL_01[['Indicador','Variante','Fecha']] = 'IS01','ZMM_OC_OL_01',Fecha
ZMM_OC_RS_01[['Indicador','Variante','Fecha']] = 'IS05','ZMM_OC_RS_01',Fecha
ZMM_SL_OL_01[['Indicador','Variante','Fecha']] = 'IS10','ZMM_SL_OL_01',Fecha

#Autorizaciones críticas
Z_ZF_BC_12[['Indicador','Variante','Fecha']] = 'IS24','Z_ZF_BC_12',Fecha
Z_ZF_BC_13[['Indicador','Variante','Fecha']] = 'IS25','Z_ZF_BC_13',Fecha
Z_ZF_BC_14[['Indicador','Variante','Fecha']] = 'IS23','Z_ZF_BC_14',Fecha
Z_ZF_HR_01[['Indicador','Variante','Fecha']] = 'IS27','Z_ZF_HR_01',Fecha
Z_ZF_HR_02[['Indicador','Variante','Fecha']] = 'IS28','Z_ZF_HR_02',Fecha
Z_ZF_HR_03[['Indicador','Variante','Fecha']] = 'IS29','Z_ZF_HR_03',Fecha
Z_ZF_HR_04[['Indicador','Variante','Fecha']] = 'IS30','Z_ZF_HR_04',Fecha
Z_ZF_HR_05[['Indicador','Variante','Fecha']] = 'IS31','Z_ZF_HR_05',Fecha
Z_ZF_HR_06[['Indicador','Variante','Fecha']] = 'IS32','Z_ZF_HR_06',Fecha
Z_ZF_EF_09[['Indicador','Variante','Fecha']] = 'IS33','Z_ZF_EF_09',Fecha
Z_ZF_EF_10[['Indicador','Variante','Fecha']] = 'IS34','Z_ZF_EF_10',Fecha
Z_ZF_EF_13[['Indicador','Variante','Fecha']] = 'IS35','Z_ZF_EF_13',Fecha
Z_ZF_EF_14[['Indicador','Variante','Fecha']] = 'IS36','Z_ZF_EF_14',Fecha
Z_ZF_EF_15[['Indicador','Variante','Fecha']] = 'IS37','Z_ZF_EF_15',Fecha
Z_ZF_EF_16[['Indicador','Variante','Fecha']] = 'IS38','Z_ZF_EF_16',Fecha

In [123]:
#Juntamos todos los DataFrames y luego reordenamos las columnas

detalle = pd.concat([ZBC_BC_BC_01,ZBC_BC_BC_02,ZBC_BC_BC_03,ZBC_BC_BC_04,ZBC_BC_BC_05,ZBC_BC_BC_06,
                     ZBC_BC_BC_07,ZBC_BC_BC_08,ZBC_BC_BC_09,ZFI_EF_MA_01,ZFI_EF_PA_01,ZFI_EF_PA_02,
                     ZFI_EF_PA_03,ZFI_EF_PA_04,ZFI_EF_PA_05,ZFI_EF_RM_01,ZFI_MA_PA_01,ZMM_OC_OL_01,
                     ZMM_OC_RS_01,ZMM_SL_OL_01,Z_ZF_BC_12,Z_ZF_BC_13,Z_ZF_BC_14,Z_ZF_HR_01,Z_ZF_HR_02,
                     Z_ZF_HR_03,Z_ZF_HR_04,Z_ZF_HR_05,Z_ZF_HR_06,Z_ZF_EF_09,Z_ZF_EF_10,Z_ZF_EF_13,Z_ZF_EF_14,Z_ZF_EF_15,Z_ZF_EF_16],axis=0).reset_index(drop=True).fillna('')
detalle['Nombre_completo'] = detalle['Nom.largo'] + detalle['Nombrecompleto']
detalle = detalle[['Indicador','Variante','Fecha','Usuario','Nombre_completo','Estado_calculado','Tipo']]

In [124]:
detalle

,Indicador,Variante,Fecha,Usuario,Nombre_completo,Estado_calculado,Tipo
0,IS04,ZFI_EF_PA_01,06-10-20,CFCASTILLOM,CamilaFernandaCastillo,ACTIVOS,ADiálogo
1,IS07,ZFI_EF_PA_02,06-10-20,AKBAIROSC,AnnyKatherineBairos,ACTIVOS,ADiálogo
2,IS07,ZFI_EF_PA_02,06-10-20,LVALDESC,LuisValdes,ACTIVOS,ADiálogo
3,IS08,ZFI_EF_PA_03,06-10-20,CFCASTILLOM,CamilaFernandaCastillo,ACTIVOS,ADiálogo
4,IS08,ZFI_EF_PA_03,06-10-20,PCARIQUEO,PATRICIOCARIQUEO,ACTIVOS,ADiálogo
...,...,...,...,...,...,...,...
608,IS34,Z_ZF_EF_10,06-10-20,YGFIGUEREDOA,YolianyGabrielaFigueredo,ACTIVOS,ADiálogo
609,IS34,Z_ZF_EF_10,06-10-20,YLEIVA,YasnaPaolaLeiva,ACTIVOS,ADiálogo
610,IS34,Z_ZF_EF_10,06-10-20,YMARAMBIO,YasminaAndreaMarambio,ACTIVOS,ADiálogo
611,IS34,Z_ZF_EF_10,06-10-20,YZAPATA,YisellaAndreaZapata,ACTIVOS,ADiálogo


In [125]:
# Agregamos desviaciones a la base histórica

detalle_des = pd.concat([detalle_des, detalle],axis=0).reset_index(drop=True).sort_values(['Indicador','Fecha'])

## Guardar en Excel

In [126]:
nombre_archivo = 'Monitoreo IS.xlsx'
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

base_cc.to_excel(writer, sheet_name='Combinaciones críticas')
base_ac.to_excel(writer, sheet_name='Autorizaciones críticas')
detalle_des.to_excel(writer, sheet_name='Detalle Desviaciones')

writer.save()